In [1]:
import pandas as pd

### Create inverse rates

In [65]:
df = pd.read_csv('testData.csv')
# append copy of dataframe to create inverse rates
df = (df
        .append(df)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
     )

In [66]:
for index in df[103:].index:
    # inverse rate
    df.loc[index, 'Rate'] = float(1/df.loc[index, 'Rate'])
    # create inverse numerator and denumerator names
    df.loc[index, 'Numerator1'] = df.loc[index, 'Denumerator']
    df.loc[index, 'Denumerator1'] = df.loc[index, 'Numerator']
    # reassign numerator and denumerator names
    df.loc[index, 'Numerator'] = df.loc[index, 'Numerator1']
    df.loc[index, 'Denumerator'] = df.loc[index, 'Denumerator1']

df = df.drop(columns=['index', 'Numerator1', 'Denumerator1'])
df.head(3)
df.shape

(206, 6)

### Create possible pairs in triangular arbitrage

In [86]:
pairList = []
# for every denumerator that has BTC as numerator
for i in list(df.Denumerator[df['Numerator'] == 'BTC']):
    # for every denumerator that has i as numerator
    for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != 'BTC') & (df['Denumerator'] != 'BTC')]))):
        if j in list(df.Denumerator[df['Numerator'] == 'BTC']):
            pairList.append([i,j])
len(pairList)

120

### Real data arbitrage

In [118]:
def triangularArbitrage(inputSet, firstTransaction, secondTransaction):
    # first transaction
    output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
    
    # second transaction
    output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
    
    # third transaction
    output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
    
#     if float(output3 - inputSet[0]) > 0:
#         print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])
    
    return float(output3 - inputSet[0])

# inputSet = (1, 'BTC')
# %time triangularArbitrage(inputSet, 'LTC', 'ETH')

In [119]:
%%time
suma = 0
n = 0
btcQuantity = 1000
for i in pairList:
    add = triangularArbitrage((btcQuantity, 'BTC'), i[0], i[1])
    if add > 0:
        suma += add
        btcQuantity += add
        n += 1
print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} separate transactions')

Obtained: 6835742.05 PLN by 59 separate transactions
Wall time: 375 ms
